In [1]:
import sys

# 1. 修正版本号进行安装
# numpy=1.26.4: 必须锁定这个版本
# ur=2024.2.0: 这是 2024 系列里最老的可用版本
print("⬇️ 正在执行最终修正版安装...")
!{sys.executable} -m pip install "numpy==1.26.4" "intel-openmp==2024.0.0" "mkl==2024.0.0" "intel-cmplr-lib-rt==2024.0.0" "intel-cmplr-lib-ur==2024.2.0" --no-cache-dir

print("✅ 安装指令执行完毕。")

⬇️ 正在执行最终修正版安装...
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/15.8 MB 11.2 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/15.8 MB 9.9 MB/s eta 0:00:02
   --------------- ------------------------ 6.0/15.8 MB 9.7 MB/s eta 0:00:02
   ------------------- -------------------- 7.9/15.8 MB 9.5 MB/s eta 0:00:01
   ----------------------- ---------------- 9.4/15.8 MB 9.2 MB/s eta 0:00:01
   --------------------------- ------------ 11.0/15.8 MB 9.2 MB/s eta 0:00:01
   ------------------------------- -------- 12.3/15.8 MB 8.5 MB/s eta 0:00:01
   --------------------------------- ------ 13.1/15.8 MB 8.4 MB/s eta 0:00:01
   ------------------------------------ --- 14.4/15.8 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 7.6 MB/s  0:00:02
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.9 MB 7.2 MB/s eta 0:

Reason for being yanked: critical bug fixed in 2024.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dpcpp-cpp-rt 2025.0.4 requires intel-openmp==2025.0.4, but you have intel-openmp 2024.0.0 which is incompatible.
intel-sycl-rt 2025.0.4 requires intel-cmplr-lib-rt==2025.0.4, but you have intel-cmplr-lib-rt 2024.0.0 which is incompatible.
intel-sycl-rt 2025.0.4 requires intel-cmplr-lib-ur==2025.0.4, but you have intel-cmplr-lib-ur 2024.2.0 which is incompatible.
mkl-dpcpp 2025.0.1 requires mkl==2025.0.1, but you have mkl 2024.0.0 which is incompatible.
onemkl-sycl-blas 2025.0.1 requires mkl==2025.0.1, but you have mkl 2024.0.0 which is incompatible.
onemkl-sycl-datafitting 2025.0.1 requires mkl==2025.0.1, but you have mkl 2024.0.0 which is incompatible.
onemkl-sycl-dft 2025.0.1 requires mkl==2025.0.1, but you have mkl 2024.0.0 which is incompatible.
onemkl-sy

In [1]:
import os
import sys
from pathlib import Path
import site

# 1. 定位环境
venv_root = Path(sys.prefix)
site_packages = venv_root / "Lib" / "site-packages"

# 2. 2024 版的标准路径结构 (bin)
# 我们手动列出这些路径，确保 Python 能找到 DLL
dll_paths = [
    site_packages / "intel_extension_for_pytorch" / "bin",
    site_packages / "intel_cmplr_lib_rt" / "bin", 
    site_packages / "intel_cmplr_lib_ur" / "bin", # 注意：UR 库在这里
    site_packages / "mkl" / "bin",
    site_packages / "intel_openmp" / "redist" / "intel64" / "vc14",
    site_packages / "tbb" / "redist" / "intel64" / "vc14"
]

print(f"正在扫描路径: {site_packages}")
valid_paths = 0

for p in dll_paths:
    if p.exists():
        try:
            os.add_dll_directory(str(p))
            os.environ['PATH'] = str(p) + ';' + os.environ['PATH']
            valid_paths += 1
            # print(f"加载: {p.name}") 
        except Exception as e:
            print(f"加载错误 {p}: {e}")
    else:
        # 如果安装成功，不应该出现这个警告
        if "intel_" in str(p):
            print(f"⚠️ 警告: 依然找不到路径 {p.name}")

if valid_paths == 0:
    print("❌ 严重: 没有加载任何 DLL。请确认第一步安装是否显示 Successfully installed。")

# 3. 导入
try:
    import torch
    import intel_extension_for_pytorch as ipex
    
    print("\n🎉🎉🎉 完美解决！")
    print(f"PyTorch: {torch.__version__}")
    print(f"IPEX: {ipex.__version__}")
    
    # 简单的显卡检查
    if torch.xpu.is_available():
        print(f"✅ 显卡: {torch.xpu.get_device_name(0)}")
        # 生成一个随机张量测试运算
        x = torch.ones(2, 2).to("xpu")
        print(f"   运算测试: Tensor on {x.device}")
    else:
        print("⚠️ 驱动未检测到 XPU，但环境已修好，可以运行 CPU 代码。")

except ImportError as e:
    print(f"\n❌ 导入失败: {e}")
except OSError as e:
    print(f"\n❌ DLL 加载失败: {e}")
    if "126" in str(e):
        print("提示: 如果文件都存在但依然报 126，请检查是否安装了 'Visual C++ Redistributable'")

正在扫描路径: d:\Users\JTST\Desktop\Desktop\JTST\.TECH\Generative-Transformer\ai\Lib\site-packages
⚠️ 警告: 依然找不到路径 bin
⚠️ 警告: 依然找不到路径 bin
⚠️ 警告: 依然找不到路径 vc14


d:\Users\JTST\Desktop\Desktop\JTST\.TECH\Generative-Transformer\ai\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'D:\Users\JTST\Desktop\Desktop\JTST\.TECH\Generative-Transformer\ai\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(



❌ DLL 加载失败: [WinError 126] 找不到指定的模块。 Error loading "d:\Users\JTST\Desktop\Desktop\JTST\.TECH\Generative-Transformer\ai\Lib\site-packages\intel_extension_for_pytorch\bin\intel-ext-pt-gpu.dll" or one of its dependencies.
提示: 如果文件都存在但依然报 126，请检查是否安装了 'Visual C++ Redistributable'


In [ ]:
with open(r"dataset/01.txt", "r", encoding='utf-8') as file:
    content = file.read()

'''print(len(content))
print(content[:100])'''  # Print the first 100 characters to verify content

chars=sorted(list(set(content)))
vocab_size=len(chars)
print("Vocab size:", vocab_size)
print("Characters:", ''.join(chars))

# Create mappings from characters to integers and vice versa
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
def encode(s):
    return [stoi[c] for c in s]
def decode(l):
    return ''.join(itos[i] for i in l)

print(encode("hello world"))
print(decode(encode("hello world")))

devices = "cpu"

In [ ]:
import torch
data = torch.tensor(encode(content), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])  # Print the first 100 encoded integers to verify encoding

# Split the data into training and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# Demonstrate how to create input-target pairs for training
block_size = 8  # context length for predictions
train_data[:block_size+1]

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context.tolist()} the target: {target}")


In [ ]:
torch.manual_seed(42)
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

batch_size = 4
x, y = get_batch('train')
print("inputs:")
print(x)
print("targets:")
print(y)

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx
    
# 假设你之前已经定义了 vocab_size 和获取了 x, y
model = BigramLanguageModel(vocab_size)
logits, loss = model(x, y)
print("logits shape:", logits.shape)
print("loss:", loss)
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))  # Decode the predicted tokens for verification



In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f"step {steps+1}: loss {loss.item()}")

In [ ]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))  # Decode the predicted tokens for verification